In [1]:
import pandas as pd
import numpy as np
import ssl
import certifi
import json
import statsmodels.api as sm
from urllib.request import urlopen
from tqdm import tqdm
from sqlalchemy import create_engine
from datetime import date

def fetch_trade_data_multi_hscode(db_info: dict,
                                   hs_codes: list,
                                   indicator: str,
                                   table_name: str = 'korea_monthly_trade_data') -> pd.DataFrame:
    """
    여러 HS 코드와 하나의 indicator에 해당하는 무역 데이터를 MySQL/MariaDB에서 조회
    """

    try:
        engine = create_engine(
            f"mysql+pymysql://{db_info['user']}:{db_info['password']}@{db_info['host']}:{db_info['port']}/{db_info['database']}"
        )

        # IN (...) 문자열 안전 생성
        in_clause = ', '.join(['%s'] * len(hs_codes))
        query = f"""
            SELECT *
            FROM {table_name}
            WHERE root_hs_code IN ({in_clause})
              AND indicator = %s
            ORDER BY root_hs_code, date
        """

        # 인자 순서: hs_codes + [indicator]
        params = hs_codes + [indicator]

        df = pd.read_sql(query, engine, params=params)
        df['date'] = pd.to_datetime(df['date'])

        return df

    except Exception as e:
        print(f"❌ 데이터 조회 실패: {e}")
        return pd.DataFrame()


def load_monthly_export_forecast(db_info, table_name='trade_forecast_by_month'):
    """
    수출 예측 데이터 전체를 월별 원형 그대로 불러오는 함수 (root_hs_code 조건 없이 전체)

    Parameters:
    - db_info (dict): {
        'host': DB 주소 (str),
        'port': 포트 번호 (int),
        'user': 사용자명 (str),
        'password': 비밀번호 (str),
        'database': 데이터베이스명 (str)
      }
    - table_name (str): 테이블 이름 (기본값: 'trade_forecast_by_month')

    Returns:
    - pd.DataFrame: 월별 수출 예측 원본 데이터프레임
    """
    try:
        engine = create_engine(
            f"mysql+pymysql://{db_info['user']}:{db_info['password']}@{db_info['host']}:{db_info['port']}/{db_info['database']}"
        )

        query = f"""
            SELECT *
            FROM {table_name}
            ORDER BY root_hs_code, period
        """

        df = pd.read_sql(query, con=engine)
        df = df.drop_duplicates(subset=['root_hs_code', 'period'])
        df['period'] = pd.to_datetime(df['period'])

        return df

    except Exception as e:
        print(f"❌ 월별 데이터 로딩 실패: {e}")
        return pd.DataFrame()



# 수출 예측 데이터를 불러오고 가공하는 함수 정의
def load_quarterly_export_forecast(db_info, root_hs_code, table_name='trade_forecast_by_month'):
    """
    특정 HS 코드를 기준으로 수출 예측 데이터를 분기별로 집계하고 변화율(pct_change)을 포함하여 반환하는 함수

    Parameters:
    - db_info (dict): DB 접속 정보
    - root_hs_code (str): 대상 HS 코드
    - table_name (str): DB 테이블명 (기본값: 'trade_forecast_by_month')

    Returns:
    - pd.DataFrame: 분기별 수출 예측 변화율 포함 데이터프레임
    """
    try:
        engine = create_engine(
            f"mysql+pymysql://{db_info['user']}:{db_info['password']}@{db_info['host']}:{db_info['port']}/{db_info['database']}"
        )

        query = f"""
            SELECT *
            FROM {table_name}
            WHERE root_hs_code = '{root_hs_code}'
            ORDER BY period
        """
        df = pd.read_sql(query, con=engine)
        df = df.drop_duplicates(subset=['period'])
        df['period'] = pd.to_datetime(df['period'])
        df['quarter'] = df['period'].dt.to_period('Q')
        quarterly_sum_df = df.groupby('quarter')['expDlr_forecast_12m'].sum().reset_index()
        quarterly_sum_df['quarter'] = quarterly_sum_df['quarter'].dt.to_timestamp()
        quarterly_sum_df['export_qoq_change'] = quarterly_sum_df['expDlr_forecast_12m'].pct_change(periods=1)
        quarterly_sum_df['export_yoy_change'] = quarterly_sum_df['expDlr_forecast_12m'].pct_change(periods=4)
        quarterly_sum_df['date_month'] = quarterly_sum_df['quarter'] + pd.offsets.QuarterEnd(0)
        quarterly_sum_df['root_hs_code'] = root_hs_code  # ✅ 구분자 추가
        return quarterly_sum_df

    except Exception as e:
        print(f"❌ 데이터 처리 실패: {e}")
        return pd.DataFrame()


# JSON 데이터를 불러오는 함수
def get_jsonparsed_data(url):
    context = ssl.create_default_context(cafile=certifi.where())
    with urlopen(url, context=context) as response:
        data = response.read().decode("utf-8")
        return json.loads(data)

# 주요 수치 변환용 컬럼 정의
compustat_is = ['report_date', 'ticker', 'period', 'sale', 'cogs', 'gp', 'xrd', 'xsga', 'idit', 'xint', 'dp', 'ebitda',
                'xopr', 'opiti', 'opir', 'pi', 'pir', 'txt', 'ni', 'nir', 'eps', 'epsdi', 'shrout', 'shroutdi']

## JSON 데이터를 불러오는 함수
def get_jsonparsed_data(url):
    context = ssl.create_default_context(cafile=certifi.where())
    with urlopen(url, context=context) as response:
        data = response.read().decode("utf-8")
        return json.loads(data)

# 주요 수치 변환용 컬럼 정의
compustat_is = ['report_date', 'ticker', 'period', 'sale', 'cogs', 'gp', 'xrd', 'xsga', 'idit', 'xint', 'dp', 'ebitda',
                'xopr', 'opiti', 'opir', 'pi', 'pir', 'txt', 'ni', 'nir', 'eps', 'epsdi', 'shrout', 'shroutdi']

# 진단용 로그 포함 함수
def forecast_sales_by_export_growth(ticker_list, forecast_quarters, quarterly_export_df):
    is_list = []
    results = []

    for ticker in tqdm(ticker_list, desc="Processing Tickers"):
        try:
            print(f"\n🔍 Processing {ticker}...")
            url = ("https://financialmodelingprep.com/api/v3/income-statement/{}?period=quarter&apikey=hT0gAk87j9xZx4PlBApvBqfVL5IahvgV".format(ticker))
            fs_raw = get_jsonparsed_data(url)
            print(f"✅ API 응답 수: {len(fs_raw)}")

            temp_df = pd.DataFrame(fs_raw)
            if temp_df.empty:
                print("❌ API 결과가 비어 있음.")
                continue

            is_df = temp_df[['date', 'symbol','period', 'revenue', 'costOfRevenue', 'grossProfit',
                             'researchAndDevelopmentExpenses', 'sellingGeneralAndAdministrativeExpenses',
                             'interestIncome', 'interestExpense', 'depreciationAndAmortization', 'ebitda',
                             'operatingExpenses', 'operatingIncome', 'operatingIncomeRatio', 'incomeBeforeTax',
                             'incomeBeforeTaxRatio','incomeTaxExpense', 'netIncome', 'netIncomeRatio', 'eps',
                             'epsdiluted', 'weightedAverageShsOut','weightedAverageShsOutDil']]
            is_df.columns = compustat_is
        except Exception as e:
            print(f"❌ 예외 발생: {e}")
            continue

        is_df_sorted = is_df.sort_values(by='report_date')
        is_df_sorted['report_date'] = pd.to_datetime(is_df_sorted['report_date'], errors='coerce')
        is_df_sorted['date_month'] = is_df_sorted['report_date'].dt.to_period('M').astype(str)
        is_df_sorted['date'] = pd.to_datetime(is_df_sorted['date_month']) + pd.offsets.MonthEnd(0)
        
        # 오늘 날짜 기준 2달 전 말일을 계산
        end_date = pd.Timestamp.today() - pd.DateOffset(months=2)
        end_date = end_date + pd.offsets.MonthEnd(0)
        
        # 날짜 범위 생성
        dates_list = pd.date_range('2004-01-31', end_date, freq='ME')
        date_df = pd.DataFrame(dates_list, columns=['date'])
        temp_is = pd.merge(date_df, is_df_sorted, on='date', how='left').ffill()

        target_data = temp_is[['date', 'date_month', 'ticker', 'period', 'sale', 'gp', 'ni']].copy()
        target_data_drop = target_data.drop_duplicates(subset=['date_month', 'period'], keep='last').copy()
        target_data_drop['date_month'] = pd.to_datetime(target_data_drop['date_month'])
        target_data_drop = target_data_drop.sort_values(['ticker', 'date_month']).reset_index(drop=True)

        for col in ['sale', 'gp', 'ni']:
            target_data_drop[f'{col}_yoy'] = target_data_drop.groupby('ticker')[col].transform(lambda x: x.pct_change(4))

        # 날짜 NaT 제거
        quarterly_export_df['date_month'] = pd.to_datetime(quarterly_export_df['date_month'])
        target_data_drop['date_month'] = pd.to_datetime(target_data_drop['date_month'])
        target_data_drop = target_data_drop[target_data_drop['date_month'].notna()].copy()
        quarterly_export_df = quarterly_export_df[quarterly_export_df['date_month'].notna()].copy()

        # 날짜 정렬
        target_data_drop = target_data_drop.sort_values('date_month')
        quarterly_export_df = quarterly_export_df.sort_values('date_month')

        print("📌 최근 매출 데이터:")
        print(target_data_drop[['date_month', 'sale']].tail())

        regression_table = pd.merge_asof(
            target_data_drop,
            quarterly_export_df[['date_month', 'export_yoy_change']],
            on='date_month',
            direction='nearest'
        )

        reg_df = regression_table[['export_yoy_change', 'sale_yoy']].copy()
        reg_df = reg_df.replace([np.inf, -np.inf], np.nan).dropna()

        print(f"📊 회귀분석 데이터 크기: {len(reg_df)}")
        if len(reg_df) < 5:
            print("⚠️ 유효한 회귀 데이터 부족. 건너뜀.")
            continue

        X = sm.add_constant(reg_df['export_yoy_change'])
        Y = reg_df['sale_yoy']
        model = sm.OLS(Y, X).fit()

        intercept = model.params['const']
        slope = model.params['export_yoy_change']
        p_value = model.pvalues['export_yoy_change']

        if target_data_drop['sale'].notna().sum() == 0:
            print("⚠️ 최근 매출 데이터 없음. 건너뜀.")
            continue

        last_sale = target_data_drop[target_data_drop['sale'].notna()].iloc[-1]['sale']
        current_sale = last_sale

        for f_date in forecast_quarters:
            export_yoy = quarterly_export_df[quarterly_export_df['date_month'] == pd.to_datetime(f_date)]['export_yoy_change']
            if export_yoy.empty:
                print(f"⛔ export_yoy_change 데이터 없음: {f_date}")
                continue

            export_yoy_val = export_yoy.values[0]
            predicted_growth = intercept + slope * export_yoy_val
            predicted_sale = current_sale * (1 + predicted_growth)

            results.append({
                'ticker': ticker,
                'forecated_salse': predicted_sale,
                'forecated_growth': predicted_growth,
                'slop': slope,
                'intercept': intercept,
                'p_value': p_value,
                'date_month': f_date, 
                'last_real_sale': last_sale 
            })

            current_sale = predicted_sale

        print(f"✅ {ticker} 예측 결과 {len(results)}건 누적")

    result_df = pd.DataFrame(results)
    
    # result_df_sum = result_df.groupby('ticker').apply(lambda df: df['forecated_salse'].sum() + df['last_real_sale'].iloc[0])
    
    return result_df


In [2]:
def korea_trade_data_forecast_upload(df_forecast: pd.DataFrame, db_info: dict, table_name: str = 'korea_monthly_trade_forecast_data'):
    """
    수출입 예측 데이터를 MySQL DB에 업로드하고 연결을 종료하는 함수

    Parameters:
    - df_forecast (pd.DataFrame): 업로드할 데이터프레임
    - db_info (dict): DB 접속 정보 (user, password, host, port, database)
    - table_name (str): 업로드할 테이블 이름 (기본값: 'korea_monthly_trade_forecast_data')
    """
    try:
        # SQLAlchemy 엔진 생성
        engine = create_engine(
            f"mysql+pymysql://{db_info['user']}:{db_info['password']}@{db_info['host']}:{db_info['port']}/{db_info['database']}"
        )

        # DB 업로드
        df_forecast.to_sql(name=table_name, con=engine, index=False, if_exists='replace')
        print(f"✅ {table_name} 테이블에 {len(df_forecast)}개의 행이 업로드되었습니다.")
    
    except Exception as e:
        print(f"❌ DB 업로드 중 오류 발생: {e}")
    
    finally:
        # 연결 해제
        engine.dispose()
        print("🔌 DB 연결이 정상적으로 종료되었습니다.")

In [3]:
from itertools import product
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from itertools import product
import warnings

warnings.filterwarnings("ignore")

def forecast_sarima_with_log(df: pd.DataFrame,
                              indicator: str,
                              forecast_steps: int = 12,
                              min_required_points: int = 36,
                              seasonal_order=(1, 1, 1, 12),
                              p_range=(0, 2), d_range=(0, 2), q_range=(0, 2),
                              log_transform: bool = True) -> pd.DataFrame:
    """
    SARIMA 예측 함수: 로그 변환 여부 선택 가능

    Parameters:
    - log_transform (bool): 로그 변환 여부 (True면 log→exp, False면 그대로 처리)

    Returns:
    - 예측 결과 long-format DataFrame
    """

    result_list = []
    df_filtered = df[df['indicator'] == indicator].copy()

    for code, group in df_filtered.groupby('root_hs_code'):
        ts = group.sort_values('date').set_index('date')['value'].dropna()

        if log_transform and (ts <= 0).any():
            print(f"⚠️ {code}-{indicator}: 로그 변환 불가 (음수/0 포함) → 예측 생략")
            continue

        if len(ts) < min_required_points:
            print(f"⚠️ {code}-{indicator}: 데이터 {len(ts)}개 → 예측 생략")
            continue

        if log_transform:
            ts_model = np.log(ts)
        else:
            ts_model = ts.copy()

        best_aic = np.inf
        best_model = None

        for order in product(range(*p_range), range(*d_range), range(*q_range)):
            try:
                model = SARIMAX(ts_model,
                                order=order,
                                seasonal_order=seasonal_order,
                                enforce_stationarity=False,
                                enforce_invertibility=False)
                result = model.fit(disp=False, maxiter=100)
                if result.aic < best_aic:
                    best_aic = result.aic
                    best_model = result
            except:
                continue

        if best_model is None:
            print(f"❌ {code}-{indicator}: 적합 가능한 SARIMA 없음 → 예측 생략")
            continue

        forecast_index = pd.date_range(start=ts.index[-1] + pd.offsets.MonthEnd(1),
                                       periods=forecast_steps, freq='M')
        forecast_values = best_model.forecast(steps=forecast_steps)

        if log_transform:
            forecast_values = np.exp(forecast_values)

        forecast_df = pd.DataFrame({
            'date': forecast_index,
            'root_hs_code': code,
            'indicator': indicator,
            'value': forecast_values,
            'is_forecast': 1
        })

        result_list.append(forecast_df)

    if result_list:
        return pd.concat(result_list, ignore_index=True)
    else:
        return pd.DataFrame(columns=['date', 'root_hs_code', 'indicator', 'value', 'is_forecast'])



In [4]:
db_info = {
    'host': 'hystox74.synology.me',
    'port': 3307,
    'user': 'stox7412',
    'password': 'Apt106503!~',
    'database': 'investar'
}


In [5]:
def fetch_trade_data_multi_hscode(db_info: dict,
                                   hs_codes: list,
                                   indicator: str,
                                   table_name: str = 'korea_monthly_trade_data') -> pd.DataFrame:
    """
    여러 HS 코드와 하나의 indicator에 해당하는 무역 데이터를 MySQL/MariaDB에서 조회
    """

    try:
        engine = create_engine(
            f"mysql+pymysql://{db_info['user']}:{db_info['password']}@{db_info['host']}:{db_info['port']}/{db_info['database']}"
        )

        # hs_codes를 안전하게 문자열로 포맷팅
        hs_codes_str = ', '.join(f"'{code}'" for code in hs_codes)

        query = f"""
            SELECT *
            FROM {table_name}
            WHERE root_hs_code IN ({hs_codes_str})
              AND indicator = '{indicator}'
            ORDER BY root_hs_code, date
        """

        df = pd.read_sql(query, engine)
        df['date'] = pd.to_datetime(df['date'])

        return df

    except Exception as e:
        print(f"❌ 데이터 조회 실패: {e}")
        return pd.DataFrame()

In [6]:
def add_yoy_growth(df, value_column='value', group_column='root_hs_code', date_column='date'):
    """
    root_hs_code별로 value 컬럼의 연간 증가율을 계산하여 새로운 컬럼으로 추가합니다.
    """
    df = df.copy()
    df[date_column] = pd.to_datetime(df[date_column])
    df.sort_values(by=[group_column, date_column], inplace=True)

    # YoY (12개월 전 대비 비율 변화율) 계산
    df[f'{value_column}_yoy'] = (
        df.groupby(group_column)[value_column]
        .transform(lambda x: x.pct_change(periods=12))
    )

    return df

In [11]:
## get hs code 

# DB 접속 정보 설정
db_info = {
    'user': 'stox7412',         # 예: 'root'
    'password': 'Apt106503!~', # 예: '1234'
    # 'host': '192.168.0.230',         # 예: 'localhost' 또는 IP
    'host' : 'hystox74.synology.me',
    'port': '3307',              # 기본 포트는 보통 3306
    'database': 'investar'        # 예: 'trade_data'
}

# SQLAlchemy 엔진 생성
engine = create_engine(
    f"mysql+pymysql://{db_info['user']}:{db_info['password']}@{db_info['host']}:{db_info['port']}/{db_info['database']}"
)

# 테이블 이름
table_name = 'target_hs_code'

# 고유한 hs_code 값 추출 쿼리 실행
query = f"SELECT DISTINCT hs_code FROM {table_name}"
unique_hs_codes_df = pd.read_sql(query, con=engine)
hs_codes  = unique_hs_codes_df['hs_code'].unique().tolist()

indicator = 'expDlr'

df_real = fetch_trade_data_multi_hscode(db_info, hs_codes, indicator)

In [12]:
# 실측 데이터에 is_forecast 플래그 추가
df_real['is_forecast'] = 0

# 예측 실행 (expDlr만 예측)
df_forecast = forecast_sarima_with_log(df_real, indicator='expDlr', forecast_steps=12, log_transform=True)

# 실측 + 예측 결합
df_combined = pd.concat([df_real, df_forecast], ignore_index=True)

# NaN 처리: 실측 쪽 is_forecast = 0
df_combined['is_forecast'] = df_combined['is_forecast'].fillna(0).astype(int)

# 오늘 날짜 구하기
today = date.today()
df_forecast['forecast_date'] = today.strftime('%Y-%m-%d')
df_forecast


⚠️ 210111-expDlr: 로그 변환 불가 (음수/0 포함) → 예측 생략
⚠️ 260300-expDlr: 로그 변환 불가 (음수/0 포함) → 예측 생략
⚠️ 271119-expDlr: 로그 변환 불가 (음수/0 포함) → 예측 생략
⚠️ 282520-expDlr: 로그 변환 불가 (음수/0 포함) → 예측 생략
⚠️ 283324-expDlr: 로그 변환 불가 (음수/0 포함) → 예측 생략
⚠️ 283691-expDlr: 로그 변환 불가 (음수/0 포함) → 예측 생략
⚠️ 290241-expDlr: 로그 변환 불가 (음수/0 포함) → 예측 생략
⚠️ 294130-expDlr: 로그 변환 불가 (음수/0 포함) → 예측 생략
⚠️ 360610-expDlr: 로그 변환 불가 (음수/0 포함) → 예측 생략
⚠️ 382490-expDlr: 로그 변환 불가 (음수/0 포함) → 예측 생략
⚠️ 382600-expDlr: 로그 변환 불가 (음수/0 포함) → 예측 생략
⚠️ 390220-expDlr: 로그 변환 불가 (음수/0 포함) → 예측 생략
⚠️ 400251-expDlr: 로그 변환 불가 (음수/0 포함) → 예측 생략
⚠️ 470329-expDlr: 로그 변환 불가 (음수/0 포함) → 예측 생략
⚠️ 481092-expDlr: 로그 변환 불가 (음수/0 포함) → 예측 생략
⚠️ 700319-expDlr: 로그 변환 불가 (음수/0 포함) → 예측 생략
⚠️ 700490-expDlr: 로그 변환 불가 (음수/0 포함) → 예측 생략
⚠️ 711021-expDlr: 로그 변환 불가 (음수/0 포함) → 예측 생략
⚠️ 711890-expDlr: 로그 변환 불가 (음수/0 포함) → 예측 생략
⚠️ 720712-expDlr: 로그 변환 불가 (음수/0 포함) → 예측 생략
⚠️ 722530-expDlr: 로그 변환 불가 (음수/0 포함) → 예측 생략
⚠️ 722550-expDlr: 로그 변환 불가 (음수/0 포함) → 예측 생략
⚠️ 722592-

,date,root_hs_code,indicator,value,is_forecast,forecast_date
0,2025-05-31,121120,expDlr,4.639640e+06,1,2025-06-09
1,2025-06-30,121120,expDlr,7.699255e+06,1,2025-06-09
2,2025-07-31,121120,expDlr,5.211496e+06,1,2025-06-09
3,2025-08-31,121120,expDlr,5.122741e+06,1,2025-06-09
4,2025-09-30,121120,expDlr,5.720561e+06,1,2025-06-09
...,...,...,...,...,...,...
6187,2025-12-31,970191,expDlr,9.145112e+06,1,2025-06-09
6188,2026-01-31,970191,expDlr,1.067601e+07,1,2025-06-09
6189,2026-02-28,970191,expDlr,1.248324e+07,1,2025-06-09
6190,2026-03-31,970191,expDlr,1.327368e+07,1,2025-06-09


In [17]:
# 함수 호출
# korea_trade_data_forecast_upload(df_forecast, db_info)

korea_trade_data_forecast_upload(df_combined, db_info, table_name = 'korea_monthly_trade_act_forecast_data')

✅ korea_monthly_trade_act_forecast_data 테이블에 121600개의 행이 업로드되었습니다.
🔌 DB 연결이 정상적으로 종료되었습니다.


In [18]:
# 적용
df_combined_with_yoy = add_yoy_growth(df_combined)

In [19]:
company_num = 490

# 1. 예측 데이터만 필터링하고 NaN 제거
df_forecast_only = df_combined_with_yoy[
    (df_combined_with_yoy['is_forecast'] == 1) &
    (df_combined_with_yoy['value_yoy'].notna())
]

# 2. root_hs_code별 평균 YoY 증가율 계산
avg_yoy_by_code = (
    df_forecast_only
    .groupby('root_hs_code')['value_yoy']
    .mean()
    .reset_index()
    .rename(columns={'value_yoy': 'avg_forecast_yoy'})
)

# 3. 수출 증가율 기준으로 상위 10개 추출
top_company_codes = avg_yoy_by_code.sort_values(by='avg_forecast_yoy', ascending=False).head(company_num)

top_company_codes

,root_hs_code,avg_forecast_yoy
302,841940,14.327663
390,851419,3.052754
336,845899,2.902217
476,890120,1.118835
301,841939,0.948747
...,...,...
243,740321,-0.133070
411,852990,-0.133826
167,621710,-0.134349
481,900211,-0.136224


In [23]:
# 테이블 이름
table_name = 'hs_code_by_kr_monster_company'

# SQL 쿼리 작성
query = f"SELECT * FROM {table_name}"

# 전체 데이터 조회
monsters_info = pd.read_sql(query, con=engine)

# # 결과 확인
# print(f"✅ {table_name} 테이블에서 {len(df_all)}개의 행을 가져왔습니다.")
# print(monsters_info.head())

In [22]:
monsters_info

,hs_code,품목명,Code,Name,hs_code_6d
0,392321,엘씨디카세트,A089980,상아프론테크,392321
1,470710,폐골판지,A016590,신대양제지,470710
2,480511,골심지,A016590,신대양제지,480511
3,560394,폴리우레탄합성피혁,A035150,백산,560394
4,630293,극세사클리너,A065950,웰크론,630293
...,...,...,...,...,...
180,9031809091,반도체오버레이계측기,A322310,오로스테크놀로지,903180
181,9401309000,디지털구강내엑스선영상획득장치,A228850,레이언스,940130
182,9403309000,사무용의자,A016800,퍼시스,940330
183,9405409000,선박용형광등기구,A108380,대양전기공업,940540


In [24]:
# 두 DataFrame을 hs_code / root_hs_code 기준으로 병합
merged_df = pd.merge(
    monsters_info, 
    top_company_codes, 
    left_on='hs_code', 
    right_on='root_hs_code', 
    how='inner'  # 공통된 값만 결합
)

# 결과 확인
print(f"✅ 결합된 데이터프레임의 크기: {merged_df.shape}")
print(merged_df.head())

✅ 결합된 데이터프레임의 크기: (18, 7)
  hs_code        품목명     Code    Name hs_code_6d root_hs_code  \
0  392321     엘씨디카세트  A089980  상아프론테크     392321       392321   
1  470710       폐골판지  A016590   신대양제지     470710       470710   
2  480511        골심지  A016590   신대양제지     480511       480511   
3  560394  폴리우레탄합성피혁  A035150      백산     560394       560394   
4  630293     극세사클리너  A065950     웰크론     630293       630293   

   avg_forecast_yoy  
0          0.089682  
1          0.067065  
2          0.159842  
3         -0.029331  
4          0.012888  


In [25]:
merged_df

,hs_code,품목명,Code,Name,hs_code_6d,root_hs_code,avg_forecast_yoy
0,392321,엘씨디카세트,A089980,상아프론테크,392321,392321,0.089682
1,470710,폐골판지,A016590,신대양제지,470710,470710,0.067065
2,480511,골심지,A016590,신대양제지,480511,480511,0.159842
3,560394,폴리우레탄합성피혁,A035150,백산,560394,560394,-0.029331
4,630293,극세사클리너,A065950,웰크론,630293,630293,0.012888
5,630710,극세사클리너,A065950,웰크론,630710,630710,-0.014378
6,721069,용융알루미늄도금강판,A058430,포스코강판,721069,721069,0.049588
7,841830,초저온냉동고,A068330,일신바이오,841830,841830,0.004520
8,841840,초저온냉동고,A068330,일신바이오,841840,841840,-0.064224
9,842952,굴삭기,A267270,현대건설기계,842952,842952,0.339268


In [60]:
merged_df

,hs_code,품목명,Code,Name,hs_code_6d,root_hs_code,avg_forecast_yoy
0,392321,엘씨디카세트,A089980,상아프론테크,392321,392321,0.089955
1,470710,폐골판지,A016590,신대양제지,470710,470710,0.067081
2,480511,골심지,A016590,신대양제지,480511,480511,0.159840
3,560394,폴리우레탄합성피혁,A035150,백산,560394,560394,-0.029331
4,630293,극세사클리너,A065950,웰크론,630293,630293,0.012882
5,630710,극세사클리너,A065950,웰크론,630710,630710,-0.014378
6,721069,용융알루미늄도금강판,A058430,포스코강판,721069,721069,0.049587
7,841830,초저온냉동고,A068330,일신바이오,841830,841830,0.004520
8,841840,초저온냉동고,A068330,일신바이오,841840,841840,-0.064224
9,842952,굴삭기,A267270,현대건설기계,842952,842952,0.339268


In [13]:
# 결합 테스트 (함수 실행 예시)
quarterly_export_all_df = load_quarterly_export_forecast(
    db_info={
        'host': '192.168.0.230',
        'port': 3307,
        'user': 'stox7412',
        'password': 'Apt106503!~',
        'database': 'investar'
    },
    root_hs_code = '854232'
)

In [42]:
ticker_list = ['MU', 'AMAT', 'AVGO', 'LRCX', 'KLAC', 'ASML', 'COHR', 'UCTT', 'ICHR', 'ONTO', 'FORM', 'TER']

forecast_quarters = ['2025-06-30', '2025-09-30', '2026-12-31', '2026-03-31']

forecated_result = forecast_sales_by_export_growth(ticker_list, forecast_quarters, quarterly_export_df)

forecast_sum = (
    forecated_result.groupby('ticker', group_keys=False)
    .apply(lambda df: df['forecated_salse'].sum() + df['last_real_sale'].iloc[0])
)

Processing Tickers:   0%|          | 0/12 [00:00<?, ?it/s]


🔍 Processing MU...


Processing Tickers:   8%|▊         | 1/12 [00:01<00:17,  1.58s/it]

✅ API 응답 수: 159
📌 최근 매출 데이터:
   date_month          sale
80 2024-02-01  5.824000e+09
81 2024-05-01  6.811000e+09
82 2024-08-01  7.750000e+09
83 2024-11-01  8.709000e+09
84 2025-02-01  8.053000e+09
📊 회귀분석 데이터 크기: 65
⛔ export_yoy_change 데이터 없음: 2026-12-31
✅ MU 예측 결과 3건 누적

🔍 Processing AMAT...


Processing Tickers:  17%|█▋        | 2/12 [00:03<00:15,  1.54s/it]

...

Processing Tickers:  92%|█████████▏| 11/12 [00:15<00:01,  1.33s/it]

✅ API 응답 수: 89
📌 최근 매출 데이터:
   date_month         sale
80 2024-03-01  168725000.0
81 2024-06-01  197474000.0
82 2024-09-01  207917000.0
83 2024-12-01  189483000.0
84 2025-03-01  171356000.0
📊 회귀분석 데이터 크기: 65
⛔ export_yoy_change 데이터 없음: 2026-12-31
✅ FORM 예측 결과 33건 누적

🔍 Processing TER...


Processing Tickers: 100%|██████████| 12/12 [00:16<00:00,  1.39s/it]

✅ API 응답 수: 159
📌 최근 매출 데이터:
   date_month         sale
80 2024-03-01  597539000.0
81 2024-06-01  729879000.0
82 2024-09-01  737298000.0
83 2024-12-01  752884000.0
84 2025-03-01  685680000.0
📊 회귀분석 데이터 크기: 65
⛔ export_yoy_change 데이터 없음: 2026-12-31
✅ TER 예측 결과 36건 누적



C:\Users\82108\AppData\Local\Temp\ipykernel_4236\2116331126.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  forecated_result.groupby('ticker', group_keys=False)
